# MODELOS DE PORTA A OPERADORES CON TODAS LAS VARIABLES NUMÉRICAS

En lugar de las variables del ids_basic creado, utilizaremos todas las variables numéricas del ids para ver si mejora la performance. Para coger las variables numéricas (excluyendo las no informativas, etc.), utilizaremos el código de Carlos.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from common.src.main.python.utils.hdfs_generic import *
import os

MAX_N_EXECUTORS=15
MIN_N_EXECUTORS=1
N_CORES_EXECUTOR=4
EXECUTOR_IDLE_MAX_TIME=120
EXECUTOR_MEMORY='32g'
DRIVER_MEMORY='16g'
N_CORES_DRIVER=1
MEMORY_OVERHEAD=N_CORES_EXECUTOR*2048
#QUEUE="root.datascience.normal"
QUEUE="root.BDPtenants.es.medium"

BDA_CORE_VERSION="1.0.0"

SPARK_COMMON_OPTS=os.environ.get('SPARK_COMMON_OPTS', '')
SPARK_COMMON_OPTS+=" --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
SPARK_COMMON_OPTS+=" --conf spark.shuffle.manager=tungsten-sort"
SPARK_COMMON_OPTS+="  --queue %s" % QUEUE
APP_NAME='new_portability_model'

# Dynamic allocation configuration
SPARK_COMMON_OPTS+=" --conf spark.driver.allowMultipleContexts=true"
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.enabled=true"
SPARK_COMMON_OPTS+=" --conf spark.shuffle.service.enabled=true"
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
SPARK_COMMON_OPTS+=" --conf spark.ui.port=58201"
SPARK_COMMON_OPTS+=" --conf spark.port.maxRetries=200"
SPARK_COMMON_OPTS+=" --executor-cores=%s" % (N_CORES_EXECUTOR)
SPARK_COMMON_OPTS+=" --conf spark.app.name=%s" % (APP_NAME)

BDA_ENV = os.environ.get('BDA_USER_HOME', '')

# Attach bda-core-ra codebase
SPARK_COMMON_OPTS+=" --files \
{}/scripts/properties/red_agent/nodes.properties,\
{}/scripts/properties/red_agent/nodes-de.properties,\
{}/scripts/properties/red_agent/nodes-es.properties,\
{}/scripts/properties/red_agent/nodes-ie.properties,\
{}/scripts/properties/red_agent/nodes-it.properties,\
{}/scripts/properties/red_agent/nodes-pt.properties,\
{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

#print os.environ.get('SPARK_COMMON_OPTS', '')
#print os.environ.get('PYSPARK_SUBMIT_ARGS', '')

sc, sparkSession, sqlContext = run_sc()
print sc.defaultParallelism

2


In [3]:
# This literal_eval is needed since 
# we have to read from a textfile
# which is formatted as python objects.
# It is totally safe.
from ast import literal_eval

# Standard Library stuff:
from functools import partial
from datetime import date, timedelta, datetime

# Numpy stuff
from numpy import (nan as np_nan, round as np_round, int64 as np_int64)
import numpy as np


# Spark stuff
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from pyspark.sql.functions import (udf, col, decode, when, lit, lower, upper, concat,
                                   translate, count, sum as sql_sum, max as sql_max, min as sql_min,
                                   round, 
                                   mean, stddev, datediff,
                                   length,
                                   countDistinct,
                                   hour, date_format, collect_set, collect_list,
                                   year, month, dayofmonth,
                                   rank, expr, lag, coalesce, row_number,
                                   isnull, isnan,
                                   unix_timestamp,
                                   regexp_replace
                                  )

from pyspark.sql.types import DoubleType, StringType, IntegerType, ArrayType, FloatType, StructType, StructField

from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline

from pyspark.ml import Pipeline

from pyspark.sql.functions import row_number, col

from pyspark.sql import DataFrameStatFunctions as statFunc

from pyspark.sql.window import Window

import json
from collections import OrderedDict

from subprocess import Popen, PIPE
import datetime, calendar
from pyspark.sql import functions as F
import datetime as dt

from pyspark.ml.feature import StandardScaler

In [4]:
import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import time
# Spark utils
from pyspark.sql.functions import (udf, col, decode, when, lit, lower, concat,
                                   translate, count, max, avg, min as sql_min,
                                   greatest,
                                   least,
                                   isnull,
                                   isnan,
                                   struct,
                                   substring,
                                   size,
                                   length,
                                   year,
                                   month,
                                   dayofmonth,
                                   unix_timestamp,
                                   date_format,
                                   from_unixtime,
                                   datediff,
                                   to_date,
                                   desc,
                                   asc,
                                   countDistinct,
                                   row_number,
                                   regexp_replace,
                                   lpad,
                                   rpad,
                                   trim,
                                   split,
                                   coalesce,
                                   array)
from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
# from pyspark.ml import Pipeline
# from pyspark.ml.classification import RandomForestClassifier
# from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import datetime as dt

In [5]:
spark = (SparkSession.builder
         .master("yarn")
         .config("spark.submit.deployMode", "client")
         .config("spark.ui.showConsoleProgress", "true")
         .enableHiveSupport()
         .getOrCreate()
         )

# sc = spark.sparkContext

In [6]:
%load_ext autoreload
%autoreload 2

#import re
import subprocess
#import sys
import time

from IPython.display import HTML, display
import tabulate

def printHTML(df, sample=7):
    display(HTML(tabulate.tabulate([df.columns]+df.take(sample), tablefmt='html', headers='firstrow')))
    
# Spark utils
from pyspark.sql.functions import (array_contains, bround, col, collect_set, concat, count, decode, desc, 
                                   isnull, length, lit, lower, lpad, max as sql_max, 
                                   size, struct, substring, sum as sql_sum, 
                                   translate, trim, udf, upper, when
                                  )
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructField, StructType

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
USECASES_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "repositorios", "use-cases")
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)

In [8]:
PYKHAOS_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "repositorios")
if PYKHAOS_SRC not in sys.path: 
    sys.path.append(PYKHAOS_SRC)

### Escojo las variables numéricas  

In [9]:
sys.path.append("/var/SP/data/home/carnaum2/ids/amdocs_inf_dataset")

In [10]:
from src.main.python.configuration.constants import ENVIRONMENT
from src.main.python.utils.spark_creator import SparkCreator
from src.main.python.pipelines.billing import Billing
from src.main.python.pipelines.breakdowns import Breakdowns
from src.main.python.pipelines.call_centre_calls import CallCentreCalls
from src.main.python.pipelines.campaigns import Campaigns
from src.main.python.pipelines.claims import Claims
from src.main.python.pipelines.competitors_web import CompWeb
from src.main.python.pipelines.customer import Customer
from src.main.python.pipelines.customer_aggregations import Customer_Aggregations
from src.main.python.pipelines.penalties import PenaltiesCustomer, PenaltiesServices
from src.main.python.pipelines.device_catalogue import Device_Catalogue
from src.main.python.pipelines.geneva_traffic import GenevaVoiceTypeUsage
from src.main.python.pipelines.geneva_traffic import GenevaVoiceUsage
from src.main.python.pipelines.geneva_traffic import GenevaRoamVoiceUsage
from src.main.python.pipelines.geneva_traffic import GenevaDataUsage
from src.main.python.pipelines.geneva_traffic import GenevaRoamDataUsage
from src.main.python.pipelines.mobile_spinners_extractor import Mobile_spinners_extractor
from src.main.python.pipelines.netscout import Netscout
from src.main.python.pipelines.orders import Orders
from src.main.python.pipelines.orders_aggregations import OrdersAgg
from src.main.python.pipelines.permsandprefs import Perms_and_prefs
from src.main.python.pipelines.services import Services
from src.main.python.pipelines.services_problems import ServiceProblems
from src.main.python.pipelines.tech_suprt import TechSupport
from src.main.python.pipelines.tgs import Tgs
from src.main.python.pipelines.tnps import Tnps
from src.main.python.pipelines.orders_sla import Orders_sla
from src.main.python.pipelines.tickets import Tickets
from src.main.python.pipelines.refund import Refund
sc = SparkCreator()
date = "20191014"
module_constructors = (Customer(sc, date, ENVIRONMENT),
                       Services(sc, date, ENVIRONMENT),
                       Customer_Aggregations(sc, date, ENVIRONMENT),
                       Billing(sc, date, ENVIRONMENT),
                       Campaigns(sc, date, date, ENVIRONMENT),
                       GenevaVoiceTypeUsage(sc, date, date, ENVIRONMENT),
                       GenevaVoiceUsage(sc, date, date, ENVIRONMENT),
                       GenevaDataUsage(sc, date, date, ENVIRONMENT),
                       #GenevaRoamVoiceUsage(sc, date, date, ENVIRONMENT),
                       #GenevaRoamDataUsage(sc, date, date, ENVIRONMENT),
                       Orders(sc, date, date, ENVIRONMENT),
                       OrdersAgg(sc, date, date, ENVIRONMENT),
                       PenaltiesCustomer(sc, date, ENVIRONMENT),
                       PenaltiesServices(sc, date, ENVIRONMENT),
                       Device_Catalogue(sc, date, date, ENVIRONMENT),
                       Perms_and_prefs(sc, date, ENVIRONMENT),
                       CallCentreCalls(sc, date, date, ENVIRONMENT),
                       Tnps(sc, date, date, ENVIRONMENT),
                       Tgs(sc, date, ENVIRONMENT),
                       Claims(sc, date, ENVIRONMENT),
                       Breakdowns(sc, date, ENVIRONMENT),
                       TechSupport(sc, date, ENVIRONMENT),
                       Netscout(sc, date, date, ENVIRONMENT),
                       CompWeb(sc, date, date, ENVIRONMENT),
                       ServiceProblems(sc, date, ENVIRONMENT),
                       Mobile_spinners_extractor(sc, date, ENVIRONMENT),
                       Orders_sla(sc, date, ENVIRONMENT),
                       Refund(sc, date, ENVIRONMENT),
                       #Tickets(sc, date, ENVIRONMENT)
                       )
na_map = {}
for module in module_constructors:
    metadata = module.set_module_metadata()
    na_map.update(metadata)
final_map = {colmn: na_map[colmn][0] for colmn in na_map.keys()
             if colmn in na_map.keys() and na_map[colmn][1] != "id"}
categ_map = {colmn: na_map[colmn][0] for colmn in na_map.keys()
             if colmn in na_map.keys() and na_map[colmn][1] == "categorical"}
numeric_map = {colmn: na_map[colmn][0] for colmn in na_map.keys()
               if colmn in na_map.keys() and na_map[colmn][1] == "numerical"}
date_map = {colmn: na_map[colmn][0] for colmn in na_map.keys()
            if colmn in na_map.keys() and na_map[colmn][1] == "date"}

In [11]:
numeric_variables=numeric_map.keys()

In [12]:
numeric_variables.append('msisdn') #añado el msisdn para hacer el join después

### Extraigo el ids con las variables numéricas seleccionadas  

In [13]:
def ids_numeric_selection(year_, month_, day_):


    ids_completo = (spark.read.load(
            '/data/udf/vf_es/amdocs_inf_dataset/amdocs_ids_service_level/year=' + year_ + '/month=' + month_ + '/day=' + day_))

    ids_numeric=ids_completo.select(numeric_variables) #aqui cojo las variables que se han seleccionado

    return ids_numeric

#Guardo este ids para train y test y luego hago inner join on msisdn con las columnas de target que habiamos creado

In [14]:
ids_julio_numeric=ids_numeric_selection('2019','7','31')
ids_sept_numeric=ids_numeric_selection('2019','9','30') 

In [15]:
categoricas=[item[0] for item in ids_julio_numeric.dtypes if item[1].startswith('string')] #elimino estas variables que no son numericas: no deberian aparecer (carlos lo va a corregir)

In [16]:
categoricas

['tgs_ind_riesgo_o2',
 'tgs_ind_riesgo_mm',
 'tgs_ind_riesgo_mv',
 'tgs_meses_fin_dto_ok',
 'CCC_L2_bucket_1st_interaction',
 'CCC_L2_bucket_latest_interaction',
 'CCC_L2_first_interaction',
 'Cust_Agg_flag_prepaid_nc',
 'tgs_ind_riesgo_max',
 'tgs_sol_24m',
 'CCC_L2_latest_interaction',
 'tgs_tg_marta',
 'tgs_blinda_bi_pos_n12',
 'msisdn']

In [16]:
ids_julio_numeric=ids_julio_numeric.drop('tgs_ind_riesgo_o2',
 'tgs_ind_riesgo_mm',
 'tgs_ind_riesgo_mv',
 'tgs_meses_fin_dto_ok',
 'CCC_L2_bucket_1st_interaction',
 'CCC_L2_bucket_latest_interaction',
 'CCC_L2_first_interaction',
 'Cust_Agg_flag_prepaid_nc',
 'tgs_ind_riesgo_max',
 'tgs_sol_24m',
 'CCC_L2_latest_interaction',
 'tgs_tg_marta',
 'tgs_blinda_bi_pos_n12')

In [17]:
ids_sept_numeric=ids_sept_numeric.drop('tgs_ind_riesgo_o2',
 'tgs_ind_riesgo_mm',
 'tgs_ind_riesgo_mv',
 'tgs_meses_fin_dto_ok',
 'CCC_L2_bucket_1st_interaction',
 'CCC_L2_bucket_latest_interaction',
 'CCC_L2_first_interaction',
 'Cust_Agg_flag_prepaid_nc',
 'tgs_ind_riesgo_max',
 'tgs_sol_24m',
 'CCC_L2_latest_interaction',
 'tgs_tg_marta',
 'tgs_blinda_bi_pos_n12')

### Extraigo el train y test etiquetado que habíamos utilizado en el otro modelo, para etiquetar el nuevo train y test (con un inner join)

In [18]:
train_basic= spark.read.load('/data/udf/vf_es/churn/portabPropension_model/train_final_31julio')
test_basic= spark.read.load('/data/udf/vf_es/churn/portabPropension_model/test_final_30sept')

msisdn_target_train=train_basic.select('msisdn','Operador_target','masmovil','movistar','orange','otros')
msisdn_target_test=test_basic.select('msisdn','Operador_target','masmovil','movistar','orange','otros')

In [19]:
train_numeric_final=ids_julio_numeric.join(msisdn_target_train,on='msisdn',how='inner')
test_numeric_final=ids_sept_numeric.join(msisdn_target_test,on='msisdn',how='inner')

In [38]:
train_numeric_final=train_numeric_final.cache()
train_numeric_final.count()

169429

In [39]:
test_numeric_final=test_numeric_final.cache()
test_numeric_final.count()

161893

In [26]:
train_numeric_final.groupby('Operador_target').count().show()

+---------------+-----+
|Operador_target|count|
+---------------+-----+
|              1|37039|
|              3|41050|
|              4|33022|
|              2|58318|
+---------------+-----+



### Aplico el modelo sobre el nuevo train y test

### Procesamiento previo de los datos 

In [20]:
variables_elim=['msisdn','Operador_target','masmovil','movistar','orange','otros']

In [21]:
variables = [i for i in train_numeric_final.columns if i not in variables_elim] #cojo las variables predictoras para el assemble

In [22]:
len(variables)

5766

In [23]:
def indexer_assembler(df_no_transformed): 
    
    assembler = VectorAssembler(inputCols=variables, outputCol="features")

    stages = [assembler]

    pipeline = Pipeline(stages = stages)
    
    pipeline_fit = pipeline.fit(df_no_transformed)
    
    df_transformed=pipeline_fit.transform(df_no_transformed)
    
    return df_transformed

In [24]:
train_model2=indexer_assembler(train_numeric_final)
test_model2=indexer_assembler(test_numeric_final)

In [25]:
#AUC

from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pyspark.sql.functions as F


#LIFT

import utils_model
from utils_model import get_lift
getScore = udf(lambda prob: float(prob[1]), DoubleType())

#FEATURE IMPORTANCE

def ExtractFeatureImp(featureImp, dataset, featuresCol):
   list_extract = []
   for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
       list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
   varlist = pd.DataFrame(list_extract)
   varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
   return(varlist.sort_values('score', ascending = False))

import matplotlib.pyplot as plt

In [26]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier

## Estadísticas básicas para el top 15.000: del test_basic cojo las variables y las clasifico según imputación

In [27]:
test_basic=test_basic.select('msisdn','Cust_Agg_L2_fbb_fx_first_days_since_nc','Cust_Agg_L2_fixed_fx_first_days_since_nc','Cust_Agg_L2_mobile_fx_first_days_since_nc',
'Cust_Agg_L2_tv_fx_first_days_since_nc','Cust_Agg_fbb_services_nc','Cust_Agg_fixed_services_nc','Cust_Agg_mobile_services_nc',
'Cust_Agg_tv_services_nc', 'Bill_N1_Amount_To_Pay','GNV_Data_L2_total_data_volume')

In [ ]:
#Variables que se imputan con -1:

grupo1=['Cust_Agg_L2_fbb_fx_first_days_since_nc','Cust_Agg_L2_fixed_fx_first_days_since_nc','Cust_Agg_L2_mobile_fx_first_days_since_nc',
'Cust_Agg_L2_tv_fx_first_days_since_nc'] 

#Variables que se imputan con 0 o la media: los nulos no se pueden identificar

grupo2=['Cust_Agg_fbb_services_nc','Cust_Agg_fixed_services_nc','Cust_Agg_mobile_services_nc',
'Cust_Agg_tv_services_nc', 'Bill_N1_Amount_To_Pay','GNV_Data_L2_total_data_volume']

# MASMOVIL

In [ ]:
#Balanceando clases:

n=float(37039)/float(169429-37039) #proporcion que hay que coger de los que no solicitan porta a masmovil (misma que los que sí: nº clientes que van a masmovil)

train_masmovil=train_model2.filter(train_model2['Operador_target']==1).union(train_model2.filter(train_model2['Operador_target']!=1).sample(False, n,5))


In [ ]:
target='masmovil'

#model = RandomForestClassifier(featuresCol = 'features', labelCol = target, maxDepth=8, numTrees=3000)

model = GBTClassifier(featuresCol = 'features',labelCol=target, maxDepth=5,maxIter=20)

model2_masmovil = model.fit(train_masmovil)

In [ ]:
pred_masmovil_test=model2_masmovil.transform(test_model2)
pred_masmovil_test=pred_masmovil_test.withColumn("score", getScore(col("probability")).cast(DoubleType()))
pred_masmovil_test=pred_masmovil_test.orderBy('score',ascending=False)

In [ ]:
pred_masmovil_top=pred_masmovil_test.limit(15000)
pred_masmovil_top=pred_masmovil_top.select('msisdn')

In [34]:
df_masmovil=test_basic.join(pred_masmovil_top, on='msisdn',how='inner')

In [59]:
from pyspark.sql.functions import col, avg

#Para las variables del grupo 1, antes de calcular la media elimino los nulos (los -1)

means_masmovil=[]

for variable in grupo1[0:len(grupo1)]:
        
    df_masmovil_filtered=df_masmovil.filter((df_masmovil[variable]!=-1))
    
    mean=df_masmovil_filtered.agg(avg(col(variable))).collect()[0][0]
        
    means_masmovil.append(mean)
    
print(means_masmovil)

[546.7833352674945, 982.5836843272002, 1969.7710666666667, 395.87844801454986, 8033.274144270833]


In [62]:
#Para el grupo 2, no tengo que eliminar registros

means_masmovil_2=[]

for variable in grupo2[0:len(grupo2)]:

    mean=df_masmovil.agg(avg(col(variable))).collect()[0][0]
    
    means_masmovil_2.append(mean)
    
print(means_masmovil_2)

[0.5746666666666667, 0.6014666666666667, 2.0316666666666667, 0.22806666666666667, 66.23533682158777, 8033.274144270833]


# MOVISTAR

In [29]:
n=float(58318)/float(169429-58318) #proporcion que hay que coger de los que no solicitan porta a masmovil (misma que los que sí: nº clientes que van a masmovil)

train_movistar=train_model2.filter(train_model2['Operador_target']==2).union(train_model2.filter(train_model2['Operador_target']!=2).sample(False, n,5))

In [ ]:
target='movistar'
#model = RandomForestClassifier(featuresCol = 'features', labelCol = target, maxDepth=8, numTrees=3000)
model = GBTClassifier(featuresCol = 'features',labelCol=target, maxDepth=5,maxIter=20)
model2_movistar = model.fit(train_movistar)

In [ ]:
pred_movistar_test=model2_movistar.transform(test_model2)
pred_movistar_test=pred_movistar_test.withColumn("score", getScore(col("probability")).cast(DoubleType()))
pred_movistar_test=pred_movistar_test.orderBy('score',ascending=False)

In [ ]:
pred_movistar_top=pred_movistar_test.limit(15000)

In [ ]:
pred_movistar_top=pred_movistar_top.select('msisdn')

In [ ]:
df_movistar=test_basic.join(pred_movistar_top, on='msisdn',how='inner')

In [ ]:
from pyspark.sql.functions import col, avg

#Para las variables del grupo 1, antes de calcular la media elimino los nulos (los -1)

means_movistar=[]

for variable in grupo1[0:len(grupo1)]:
        
    df_movistar_filtered=df_movistar.filter((df_movistar[variable]!=-1))
    
    mean=df_movistar_filtered.agg(avg(col(variable))).collect()[0][0]
        
    means_movistar.append(mean)
    
print(means_movistar)

In [ ]:
#Para el grupo 2, no tengo que eliminar registros

means_movistar_2=[]

for variable in grupo2[0:len(grupo2)]:

    mean=df_movistar.agg(avg(col(variable))).collect()[0][0]
    
    means_movistar_2.append(mean)
    
print(means_movistar_2)

# ORANGE

In [ ]:
n=float(41050)/float(169429-41050) #proporcion que hay que coger de los que no solicitan porta a masmovil (misma que los que sí: nº clientes que van a masmovil)

train_orange=train_model2.filter(train_model2['Operador_target']==3).union(train_model2.filter(train_model2['Operador_target']!=3).sample(False, n,5))

In [ ]:
target='orange'
#model = RandomForestClassifier(featuresCol = 'features', labelCol = target, maxDepth=8, numTrees=3000)
model = GBTClassifier(featuresCol = 'features',labelCol=target, maxDepth=5,maxIter=20)
model2_orange = model.fit(train_orange)

In [ ]:
pred_orange_test=model2_orange.transform(test_model2)
pred_orange_test=pred_orange_test.withColumn("score", getScore(col("probability")).cast(DoubleType()))
pred_orange_test=pred_orange_test.orderBy('score',ascending=False)

In [ ]:
pred_orange_top=pred_orange_test.limit(15000)
pred_orange_top=pred_orange_top.select('msisdn')

In [ ]:
df_orange=test_basic.join(pred_orange_top, on='msisdn',how='inner')

In [ ]:
#Para las variables del grupo 1, antes de calcular la media elimino los nulos (los -1)

means_orange=[]

for variable in grupo1[0:len(grupo1)]:
        
    df_orange_filtered=df_orange.filter((df_orange[variable]!=-1))
    
    mean=df_orange_filtered.agg(avg(col(variable))).collect()[0][0]
        
    means_orange.append(mean)
    
print(means_orange)

In [ ]:
#Para el grupo 2, no tengo que eliminar registros

means_orange_2=[]

for variable in grupo2[0:len(grupo2)]:

    mean=df_orange.agg(avg(col(variable))).collect()[0][0]
    
    means_orange_2.append(mean)
    
print(means_orange_2)

# OTROS

In [ ]:
n=float(33022)/float(169429-33022) #proporcion que hay que coger de los que no solicitan porta a masmovil (misma que los que sí: nº clientes que van a masmovil)

train_otros=train_model2.filter(train_model2['Operador_target']==4).union(train_model2.filter(train_model2['Operador_target']!=4).sample(False, n,5))

In [ ]:
target='otros'
#model = RandomForestClassifier(featuresCol = 'features', labelCol = target, maxDepth=8, numTrees=3000)
model = GBTClassifier(featuresCol = 'features',labelCol=target, maxDepth=5,maxIter=20)
model2_otros = model.fit(train_otros)

In [ ]:
pred_otros_test=model2_otros.transform(test_model2)
pred_otros_test=pred_otros_test.withColumn("score", getScore(col("probability")).cast(DoubleType()))
pred_otros_test=pred_otros_test.orderBy('score',ascending=False)

In [ ]:
pred_otros_top=pred_otros_test.limit(15000)
pred_otros_top=pred_otros_top.select('msisdn')

In [ ]:
df_otros=test_basic.join(pred_otros_top, on='msisdn',how='inner')

In [ ]:
#Para las variables del grupo 1, antes de calcular la media elimino los nulos (los -1)

means_otros=[]

for variable in grupo1[0:len(grupo1)]:
        
    df_otros_filtered=df_otros.filter((df_otros[variable]!=-1))
    
    mean=df_otros_filtered.agg(avg(col(variable))).collect()[0][0]
        
    means_otros.append(mean)
    
print(means_otros)

In [ ]:
#Para el grupo 2, no tengo que eliminar registros

means_otros_2=[]

for variable in grupo2[0:len(grupo2)]:

    mean=df_otros.agg(avg(col(variable))).collect()[0][0]
    
    means_otros_2.append(mean)
    
print(means_otros_2)